In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

import shutil 
from tqdm import tqdm

train_dir = 'train_temp'
val_dir = 'val'
train_dir_final = 'train'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name)             
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))
    os.makedirs(os.path.join(train_dir_final, class_name), exist_ok=True)    
        

    
!ls train

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import random
from PIL import Image


from torchvision import transforms, models
from torchvision.transforms import functional
from torch.utils.data import ConcatDataset
from matplotlib.animation import FuncAnimation
batch_size = 8

['platesv2']
After zip extraction:
['__MACOSX', '.virtual_documents', 'plates']
['test', 'train', '.DS_Store']


100%|██████████| 21/21 [00:00<00:00, 4714.21it/s]


cleaned  dirty


In [2]:
def generate_augmented_data(dir_path_temp,dir_path_final, iteration_number):
    list_of_train_datasets = []
    for i in range(iteration_number):
        train_transforms = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=(0.5,1.5), contrast=(1), saturation=(0.5,1.5), hue=(-0.1,0.1))         
            ])
        train_dataset = torchvision.datasets.ImageFolder(dir_path_temp,train_transforms)
        list_of_train_datasets.append(train_dataset)
     
    for iteration_number,each_element_in_list in enumerate(list_of_train_datasets):
        for index, each_element in enumerate(each_element_in_list): 
            if each_element[1] == 0:
                each_element[0].save(dir_path_final+'cleaned/'+str(iteration_number) + str(index)+'.png')
            else:
                each_element[0].save(dir_path_final+'dirty/'+str(iteration_number) + str(index)+'.png')
    

In [3]:
for i in range(1):
        train_transforms = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=(0.5,1.5), contrast=(1), saturation=(0.5,1.5), hue=(-0.1,0.1))         
            ])
        train_dataset = torchvision.datasets.ImageFolder('/kaggle/working/train_temp/',train_transforms)

In [4]:
generate_augmented_data('/kaggle/working/train_temp/','/kaggle/working/train/',2)


In [5]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolov8x-cls.pt")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 744.9 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 kB 6.9 MB/s eta 0:00:00a 0:00:01


100%|██████████| 110M/110M [00:00<00:00, 234MB/s]  


In [6]:
results = model.train(data="/kaggle/working/", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/kaggle/working/, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

2024-07-31 15:31:38,413	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-31 15:31:39,309	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=1000 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralyti

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all       0.75          1



100 epochs completed in 0.154 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train/weights/best.pt, 112.5MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.70 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,125,762 parameters, 0 gradients, 153.8 GFLOPs
train: /kaggle/working/train... found 64 images in 2 classes ✅ 
val: /kaggle/working/val... found 8 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00,  6.08it/s]


                   all      0.875          1
Speed: 0.5ms preprocess, 17.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


lr/pg0,▁▂▃▄▅▅▆▆▇████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
lr/pg1,▁▂▃▄▅▅▆▆▇████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
lr/pg2,▁▂▃▄▅▅▆▆▇████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
metrics/accuracy_top1,▁▆▃▆▆██▆▃▁▆▆▆▃▃▃▆▆▆▆▆▆▆▃▃▁▃▃▃▃▃▃▃▃▆▃▆▆▆█
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,██▆▅▄▃▂▂▁▁▁▁▂▁▂▁▂▁▁▁▁▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,▅▅▅▅▃▁▁▂▆▆▁▃▂▃▄▃▂▃▃▄▄▃▃▄▆█▇▇▆▆▄▄▄▄▂▁▁▂▁▂
lr/pg0,1e-05


In [28]:
res = model.predict('/kaggle/working/val/cleaned/0016.jpg')
res[0].probs


image 1/1 /kaggle/working/val/cleaned/0016.jpg: 640x640 dirty 0.86, cleaned 0.14, 61.6ms
Speed: 14.5ms preprocess, 61.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Probs object with attributes:

data: tensor([0.1362, 0.8638], device='cuda:0')
orig_shape: None
shape: torch.Size([2])
top1: 1
top1conf: tensor(0.8638, device='cuda:0')
top5: [1, 0]
top5conf: tensor([0.8638, 0.1362], device='cuda:0')